# Training "Dense Passage Retrieval" Model

using haystack

## Installing Haystack

`pip`를 이용한 Haystack 설치

In [3]:
%%bash

pip install --upgrade pip
pip install farm-haystack[colab,inference,metrics]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 8.5 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.7/10.7 MB 49.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.7/48.7 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.5/121.5 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 101.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 6.4 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 72.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ━━━━━━━━

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires openai, which is not installed.


In [4]:
pip install tensorflow_probability==0.12.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 21.4 MB/s eta 0:00:00
  Attempting uninstall: tensorflow_probability
    Found existing installation: tensorflow-probability 0.22.0
    Uninstalling tensorflow-probability-0.22.0:
      Successfully uninstalled tensorflow-probability-0.22.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
dopamine-rl 4.0.6 requires tensorflow-probability>=0.13.0, but you have tensorflow-probability 0.12.2 which is incompatible.


### Enabling Telemetry
[Telemetry](https://docs.haystack.deepset.ai/docs/telemetry) 자세한 내용 참고

In [5]:
from haystack.telemetry import tutorial_running

tutorial_running(9)

## Logging

haystack의 로깅 설정


In [6]:
import logging

logging.basicConfig(format="%(levelname)s - %(name)s -  %(message)s", level=logging.WARNING)
logging.getLogger("haystack").setLevel(logging.INFO)

In [7]:
# Here are some imports that we'll need

from haystack.nodes import DensePassageRetriever
from haystack.utils import fetch_archive_from_http
from haystack.document_stores import InMemoryDocumentStore

## Training Data

DPR 모델을 훈련하려면 질문과 관련 문서의 쌍을 만들어야 한다.

모델을 훈련하기 위해서는 원본 DPR 훈련 데이터와 동일한 형식의 데이터셋이 필요하다. 데이터셋의 각 데이터는 다음과 같은 구조를 가져야 합니다.

``` python
    {
        "dataset": str,
        "question": str,
        "answers": list of str
        "positive_ctxs": list of dictionaries of format {'title': str, 'text': str, 'score': int, 'title_score': int, 'passage_id': str}
        "negative_ctxs": list of dictionaries of format {'title': str, 'text': str, 'score': int, 'title_score': int, 'passage_id': str}
        "hard_negative_ctxs": list of dictionaries of format {'title': str, 'text': str, 'score': int, 'title_score': int, 'passage_id': str}
    }
```

`positive_ctxs`는 쿼리에 대한 정답이다. 어떤 데이터셋에서는 쿼리에 대해 여러 개의 positive 문맥이 있을 수 있으므로 `num_positives` 매개변수를 기본값 1보다 높게 설정할 수 있다. 그러나  `num_positives` 는 데이터의 각 쿼리에 대한 최소`positive_ctxs` 수 이하이어야 한다. positive 문맥이 예제당 불균형하게 있는 경우 답을 포함하는 비슷한 문맥을 검색하여 일부 소프트 레이블을 생성하는 것이 좋다.

DPR은 표준적으로 in-batch negatives라고 알려진 방법을 사용하여 훈련된다. 이는 특정 쿼리의 positive 문맥이 배치 내 다른 쿼리에 대한 negative 문맥으로 취급된다는 것을 의미한다. 이렇게 함으로써 모델이 대규모 데이터에 대해 훈련될 수 있도록 높은 정도의 계산 효율성을 달성할 수 있다.

`negative_ctxs` 는 Haystack의 DPR 훈련에서 실제로 사용되지 않으므로 빈 목록으로 설정하는 것이 좋다. 이들은 원래의 DPR 저자들에 의해 in-batch negatives 방법과 비교하기 위한 실험에서 사용되었다.

`hard_negative_ctxs` 는 쿼리와 관련이 없는 passage이다. 원래의 DPR 논문에서 이러한 passage는 쿼리에 가장 관련 있는 passage를 찾기 위해 리트리버를 사용하여 가져온다. 답변 텍스트를 포함하는 passage는 필터링된다.

## Using Question Answering Data

Question Answering datasets은 훈련 데이터로 사용될 수 있다. Google의 Natural Questions 데이터셋은 충분히 크며 충분히 고유한 passage를 포함하고 있어 DPR 훈련 세트로 변환할 수 있다. 이를 단순히 답을 포함하는 passage를 쿼리에 대한 관련 문서로 간주하여 수행한다.

## Download Original DPR Training Data

In [8]:
# Download original DPR data
# WARNING: the train set is 7.4GB and the dev set is 800MB

doc_dir = "data/tutorial9"

s3_url_train = "https://dl.fbaipublicfiles.com/dpr/data/retriever/biencoder-nq-train.json.gz"
s3_url_dev = "https://dl.fbaipublicfiles.com/dpr/data/retriever/biencoder-nq-dev.json.gz"

fetch_archive_from_http(s3_url_train, output_dir=doc_dir + "/train")
fetch_archive_from_http(s3_url_dev, output_dir=doc_dir + "/dev")

INFO:haystack.utils.import_utils:Fetching from https://dl.fbaipublicfiles.com/dpr/data/retriever/biencoder-nq-train.json.gz to 'data/tutorial9/train'
INFO:haystack.utils.import_utils:Fetching from https://dl.fbaipublicfiles.com/dpr/data/retriever/biencoder-nq-dev.json.gz to 'data/tutorial9/dev'


True

## Option 1: Training DPR from Scratch

passage 및 query embedding model 모두 BERT base를 사용하여 초기화되며 모델은 Google의 Natural Questions 데이터셋으로 훈련된다
(DPR에 특화된 형식으로).

In [9]:
# Here are the variables to specify our training data, the models that we use to initialize DPR
# and the directory where we'll be saving the model

train_filename = "/content/dpr_train_neg_yh.json"
dev_filename = "/content/dpr_train_neg_yh.json"

query_model = "kykim/bert-kor-base"
passage_model = "kykim/bert-kor-base"

save_dir = "../saved_models/dpr"

## Option 2: Finetuning DPR

데이터 셋에 맞추어 사전훈련된 bert model을 finr-tuning하기

In [10]:
# Here are the variables you might want to use instead of the set above
# in order to perform pretraining

# doc_dir = "PATH_TO_YOUR_DATA_DIR"
# train_filename = "TRAIN_FILENAME"
# dev_filename = "DEV_FILENAME"

# query_model = "facebook/dpr-question_encoder-single-nq-base"
# passage_model = "facebook/dpr-ctx_encoder-single-nq-base"

# save_dir = "../saved_models/dpr"

# Here are the variables to specify our training data, the models that we use to initialize DPR
# and the directory where we'll be saving the model

train_filename = "/content/dpr_train_neg_yh.json"
dev_filename = "/content/dpr_train_neg_yh.json"

query_model = "kykim/bert-kor-base"
passage_model = "kykim/bert-kor-base"

save_dir = "../saved_models/dpr"

## Initialization


모델을 훈련하려면 일반 언어 모델 가중치로 초기화하거나 파인튜닝을 위해 사전 훈련된 DPR 가중치로 초기화할 수 있다. 원본 DPR 매개변수를 따르되 통과 길이의 최대값은 그대로 유지하고 쿼리 길이의 최대값은 64로 설정한다. 이는 쿼리가 매우 길지 않기 때문이다.

In [11]:
## Initialize DPR model

retriever = DensePassageRetriever(
    document_store=InMemoryDocumentStore(),
    query_embedding_model=query_model,
    passage_embedding_model=passage_model,
    max_seq_len_query=64,
    max_seq_len_passage=256,
)

INFO:haystack.modeling.utils:Using devices: CPU - Number of GPUs: 0
INFO:haystack.modeling.utils:Using devices: CPU - Number of GPUs: 0


/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
INFO:haystack.modeling.model.language_model:Auto-detected model language: english
INFO:haystack.modeling.model.language_model:Auto-detected model language: english


## Training

훈련을 시작하는 코드.

V100 GPU에서는 배치 크기 16까지 맞출 수 있으므로 그래디언트 누적 단계를 8로 설정하여 원래 DPR 실험의 배치 크기 128을 시뮬레이션한다.

embed_title=True일 때 문서 제목은 입력 텍스트 시퀀스 앞에 [SEP] 토큰과 함께 추가된다.

In [12]:
# Start training our model and save it when it is finished

retriever.train(
    data_dir='/content/dpr_train_neg_yh.json)',
    train_filename=train_filename,
    dev_filename=dev_filename,
    test_filename=dev_filename,
    n_epochs=2,
    batch_size=16,
    grad_acc_steps=8,
    save_dir=save_dir,
    evaluate_every=3000,
    embed_title=False,
    num_positives=1,
    num_hard_negatives=1,
)

INFO:haystack.modeling.data_handler.data_silo:
Loading data into the data silo ... 
              ______
               |o  |   !
   __          |:`_|---'-.
  |__|______.-/ _ \-----.|
 (o)(o)------'\ _ /     ( )
 
INFO:haystack.modeling.data_handler.data_silo:LOADING TRAIN DATA
INFO:haystack.modeling.data_handler.data_silo:==================
INFO:haystack.modeling.data_handler.data_silo:Loading train set from: /content/dpr_train_neg_yh.json 
Preprocessing dataset: 100%|██████████| 3/3 [00:01<00:00,  2.59 Dicts/s]
ERROR:haystack.modeling.data_handler.processor:Unable to convert 3 samples to features. Their ids are : 336, 89, 262
INFO:haystack.modeling.data_handler.data_silo:
INFO:haystack.modeling.data_handler.data_silo:LOADING DEV DATA
INFO:haystack.modeling.data_handler.data_silo:=================
INFO:haystack.modeling.data_handler.data_silo:Loading dev set from: /content/dpr_train_neg_yh.json
Preprocessing dataset: 100%|██████████| 3/3 [00:01<00:00,  2.57 Dicts/s]
ERROR:haystack.mod

## Loading

In [13]:
reloaded_retriever = DensePassageRetriever.load(load_dir=save_dir, document_store=None)

INFO:haystack.modeling.utils:Using devices: CPU - Number of GPUs: 0
INFO:haystack.nodes.retriever.dense:DPR model loaded from ../saved_models/dpr


In [14]:
from haystack.document_stores import InMemoryDocumentStore
import pandas as pd
#import json

document_store = InMemoryDocumentStore()

#train = json.load(open(train_filename))
#dicts = [{'content':j} for j in set([i['answers'] for i in train])]

train = pd.read_csv('/content/질의데이터_yh.csv')
dicts = [{'content': j['place_info'],'meta':{'cor':j['coordinates (y, x)']}} for j in [d for idx, d in train.iterrows()]]

document_store.write_documents(dicts)
document_store.update_embeddings(reloaded_retriever)

INFO:haystack.modeling.utils:Using devices: CPU - Number of GPUs: 0
INFO:haystack.document_stores.memory:Updating embeddings for 0 docs ...
Create embeddings: 100%|██████████| 128/128 [00:07<00:00, 21.27 Docs/s]
Documents Processed: 10000 docs [00:07, 1306.88 docs/s]


In [20]:
results = reloaded_retriever.retrieve("농대 본관 수국정원 길에서 모이도록 하겠습니다.", document_store=document_store)
for i in range(3):
  print('top '+str(i+1)+', 장소: '+results[i].content+', coordinate: '+results[i].meta['cor'])

top 1, 장소: 농대(농생대) 본관 수국정원 지름길, coordinate: 35.848919, 127.132750
top 2, 장소: 농대(농생대) 공터와 캠퍼스 텃밭이 이어지는 언덕, coordinate: 35.849183, 127.129787
top 3, 장소: 농대(농생대) 공터와 캠퍼스 텃밭이 이어지는 지름길, coordinate: 35.849183, 127.129787
